In [1]:
import os
import re
import torch
import json
import nltk
import unicodedata
import pandas as pd
import numpy as np

import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from datasets import Dataset
from typing import List, Tuple
from transformers import AutoTokenizer
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split

from transformers import (
    AutoConfig,
    AutoTokenizer,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorForTokenClassification
)

C:\Users\lamem\anaconda3\Anaconda3\envs\mllmrc\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Cuda and stuff

In [ ]:
# Check whether CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
x = torch.rand(3, 3).to(device)
print(f'Tensor is on: {x.device}')

In [ ]:
# print cuda version
print(torch.version.cuda)

In [2]:
# check the current working directory
os.chdir('C:/Users/lamem/OneDrive/Documents/GHD/Microsoft-Learn-Location-Mention-Recognition-Challenge')
print(os.getcwd())

C:\Users\lamem\OneDrive\Documents\GHD\Microsoft-Learn-Location-Mention-Recognition-Challenge


In [3]:
new = pd.read_csv("lewa/Train_1.csv")
new.head()

,tweet_id,text,location
0,ID_1001136212718088192,NaN,EllicottCity
1,ID_1001136696589631488,"Flash floods struck a Maryland city on Sunday,...",Maryland
2,ID_1001136950345109504,State of emergency declared for Maryland flood...,Maryland
3,ID_1001137334056833024,Other parts of Maryland also saw significant d...,Baltimore Maryland
4,ID_1001138374923579392,"Catastrophic Flooding Slams Ellicott City, Mar...",Ellicott City Maryland


In [4]:
# Load your  
df_t = new.copy() 
df_t = df_t.dropna(subset = "text")
df_t.head()

,tweet_id,text,location
1,ID_1001136696589631488,"Flash floods struck a Maryland city on Sunday,...",Maryland
2,ID_1001136950345109504,State of emergency declared for Maryland flood...,Maryland
3,ID_1001137334056833024,Other parts of Maryland also saw significant d...,Baltimore Maryland
4,ID_1001138374923579392,"Catastrophic Flooding Slams Ellicott City, Mar...",Ellicott City Maryland
5,ID_1001138377717157888,WATCH: 1 missing after flash #FLOODING devasta...,Ellicott City Maryland


In [5]:
df_t.shape

(16448, 3)

In [6]:
# separate data into train and dev
trainData, devData = train_test_split(df_t, test_size=0.15, random_state=42)

# save data to a csv tainData
trainData.to_csv('lewa/Train-dropna.csv', index=False)

# save data to a csv devData
devData.to_csv('lewa/Dev-dropna.csv', index=False)

In [7]:
import csv
import json

def csv_to_json(csv_file_path, json_file_path):
    # Open the CSV file and create a list of dictionaries
    with open(csv_file_path, mode='r', encoding='utf-8') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        data = [row for row in csv_reader]

    # Write the list of dictionaries to a JSON file
    with open(json_file_path, mode='w', encoding='utf-8') as json_file:
        json.dump(data, json_file, indent=4)

# Example usage
#csv_file_path = 'testData/Train-dropna.csv'  # Path to your CSV file
#json_file_path = 'testData/output_file.json'  # Path where the JSON will be saved
csv_to_json('lewa/Train-dropna.csv', 'lewa/train_file.json')
csv_to_json('lewa/Dev-dropna.csv', 'lewa/dev_file.json')


IDENTIFY location from sentences 

In [10]:
import spacy
import string

# Load spaCy's English model
nlp = spacy.load("en_core_web_lg")

def remove_hashtag(text):
    # Find hashtags and process each one
    return re.sub(r'#(\w+)', lambda m: ' '.join(re.findall(r'[A-Z][^A-Z]*|[a-z]+', m.group(1))), text)

# Function to clean and preprocess text (tokenization, normalization, lemmatization)
def preprocess_text(text):
    # Remove special characters (keeping alphanumeric characters and spaces)
    text = remove_hashtag(text)  # Remove hashtags and separate words
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)

    doc = nlp(text)
    cleaned_tokens = []
    for token in doc:
        # Normalize: Convert to lowercase and remove punctuation
        if token.text not in string.punctuation:
            cleaned_tokens.append(token.lemma_)  # Lemmatize and lowercase
    return " ".join(cleaned_tokens)

# Function to extract locations as entities after preprocessing
def extract_entities(text):
    doc = nlp(text)
    entities = []
    for ent in doc.ents:
        if ent.label_ == 'GPE':  # 'GPE' is for locations
            entities.append({
                'entity': ent.text,
                'label': ent.label_,
                'start_char': ent.start_char,
                'end_char': ent.end_char
            })
    return entities

# Function to process a JSON file
def process_json_file(file_path):
    # Load JSON data
    with open(file_path, 'r') as f:
        data = json.load(f)

    # Convert the JSON data to a DataFrame
    df = pd.DataFrame(data)

    # Preprocess text and extract entities
    df['cleaned_text'] = df['text'].apply(preprocess_text)
    df['entities'] = df['cleaned_text'].apply(extract_entities)

    return df

# Paths to your two JSON files
file_path_1 = 'lewa/train_file.json'
file_path_2 = 'lewa/dev_file.json'

# Process both files
df1 = process_json_file(file_path_1)
df2 = process_json_file(file_path_2)


# If you want to save them separately, you can use the individual DataFrames:
# Save the updated data for file1
with open('lewa/train_entities.json', 'w') as f:
    json.dump(df1.to_dict(orient='records'), f, indent=4)

# Save the updated data for file2
with open('lewa/dev_entities.json', 'w') as f:
    json.dump(df2.to_dict(orient='records'), f, indent=4)

 Merge the extracted_locations with the Existing location Column

In [11]:
# Function to normalize text case

def merge_locations(df, existing_column='location', extracted_column='entities'):
    # Ensure that the existing location column is always a list
    df[existing_column] = df[existing_column].apply(lambda x: [x] if isinstance(x, str) else [])

    # Ensure extracted locations are lists of strings
    df[extracted_column] = df[extracted_column].apply(
        lambda locs: [loc for loc in locs if isinstance(loc, str)] if isinstance(locs, list) else []
    )

    # Combine existing and extracted locations, ensuring all entries are strings
    df['merged_locations'] = df.apply(
        lambda row: list(set(row[existing_column] + row[extracted_column])), axis=1
    )

    # Optionally, drop the intermediate columns if you don't need them
    #df.drop([existing_column, extracted_column], axis=1, inplace=True)

    return df

file_path_1 = 'lewa/train_entities.json'
file_path_2 = 'lewa/dev_entities.json'

# Process both files
df1 = process_json_file(file_path_1)
df2 = process_json_file(file_path_2)

df1 = merge_locations(df1)
df2 = merge_locations(df2)

# Remove duplicates within each list (if needed)
df1['merged_locations'] = df1['merged_locations'].apply(lambda locs: list(set(locs)))
df2['merged_locations'] = df2['merged_locations'].apply(lambda locs: list(set(locs)))

# Save the merged data for file1
with open('lewa/train_merged_entities.json', 'w') as f:
    json.dump(df1.to_dict(orient='records'), f, indent=4)

# Save the merged data for file2
with open('lewa/dev_merged_entities.json', 'w') as f:
    json.dump(df2.to_dict(orient='records'), f, indent=4)

BIOES TAGGING